In [1]:
import salem 
import xarray as xr
import pandas as pd
import xesmf as xe
import matplotlib.pyplot as plt
import glob
import numpy as np

In [2]:
#list(ds0.variables)

k1=['swdown']
k0=['height',
 'pressure',
 'pblh',
 'rh',
 't',
 'u',
 'v',
 #'psfc'
   ]

In [5]:

# 定义全局变量来存储 regridder
global_regridder = None

def ext(l0, l1, lon0, lon1, lat0, lat1,bottom_top=3):
    
    ## 创建经纬度网格 1d
    dx=0.1
    ds_out = xr.Dataset(
        {
            "lat": ("lat", np.arange( lat0,lat1, dx)),
            "lon": ("lon", np.arange(lon0,lon1, dx))
        }
    )

    ## 创建转换器
    #regridder = xe.Regridder(ds, ds_out, 'bilinear')
    #ds_out = regridder(ds)
    global global_regridder

    print(l0, l1)
    ds0 = xr.open_dataset(l0)
    ds1 = xr.open_dataset(l1)

    ds0 = ds0[k0].assign_coords(coords=dict(
        lon=(["south_north", "west_east"], ds0.xlon.values),
        lat=(["south_north", "west_east"], ds0.xlat.values),
        Time=pd.to_datetime(ds0.Times.values.astype(str), format="%Y-%m-%d_%H:%M:%S")
    )).sel(bottom_top=bottom_top)

    ds1 = ds1[k1].assign_coords(coords=dict(
        lon=(["south_north", "west_east"], ds1.xlon.values),
        lat=(["south_north", "west_east"], ds1.xlat.values),
        Time=pd.to_datetime(ds1.Times.values.astype(str), format="%Y-%m-%d_%H:%M:%S")
    ))  # .sel(bottom_top=0)

    ds = xr.merge([ds1, ds0])
    ds1.close()
    ds0.close()
    del ds1, ds0

    # 如果 global_regridder 为 None，创建新的 regridder，并存储为全局变量
    if global_regridder is None:
        global_regridder = create_regridder(ds, ds_out)

    # 使用全局 regridder
    ds_reg = global_regridder(ds)
    return ds_reg.sel(lon=slice(lon0, lon1), lat=slice(lat0, lat1))

def create_regridder(ds, ds_out):
    # Assuming you have xesmf library installed
    regridder = xe.Regridder(ds, ds_out, 'bilinear')
    return regridder

In [9]:
lat00=36.20
lon00=117.08
lon0,lon1,lat0,lat1=lon00-2,lon00+2,lat00-2,lat00+2
#119°08'~120°12’，北纬31°09'~32°04’
lon0,lon1,lat0,lat1

(115.08, 119.08, 34.2, 38.2)

In [10]:
day1='2021M07'
day2='2021M08'

L0=glob.glob(f'/exports/d1/wrf_only_daily/fnl1.0/{day1}/*_physical_0.nc')\
            +glob.glob(f'/exports/d1/wrf_only_daily/fnl1.0/{day2}/*_physical_0.nc')
L0.sort()
L1=glob.glob(f'/exports/d1/wrf_only_daily/fnl1.0/{day1}/*_physical_1.nc')\
            +glob.glob(f'/exports/d1/wrf_only_daily/fnl1.0/{day2}/*_physical_1.nc')
L1.sort()
len(L1),len(L0)

(62, 62)

In [11]:
#,bottom_top=3

global_regridder = None
DS=xr.concat([ext(l0,l1,lon0,lon1,lat0,lat1,bottom_top=3) for [l0,l1] in zip(L0[:40],L1[:40])],dim='Time')

/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070300_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070400_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070400_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070500_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070500_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070800_physical_

In [12]:
DS.to_netcdf('/exports/d4/xuejingkai/atmos/project2024/wrf_extract/liuchao/v2/taian_202107_202108.nc')

In [13]:
lat00=39.3
lon00=117.0
lon0,lon1,lat0,lat1=lon00-2,lon00+2,lat00-2,lat00+2
#119°08'~120°12’，北纬31°09'~32°04’
lon0,lon1,lat0,lat1

(115.0, 119.0, 37.3, 41.3)

In [25]:
from datetime import datetime, timedelta

# 初始日期
start_date = datetime.strptime('2022-07-01', '%Y-%m-%d')

# 结束日期
end_date = datetime.strptime('2022-10-31', '%Y-%m-%d')

# 存储文件路径的列表
L0 = []
L1 = []
# 循环获取每个月的文件路径
while start_date <= end_date:
    current_month = start_date.strftime('%YM%m')
    current_month_path = f'/exports/d1/wrf_only_daily/fnl1.0/{current_month}/*_physical_0.nc'
    current_month_path2 = f'/exports/d1/wrf_only_daily/fnl1.0/{current_month}/*_physical_1.nc'
    
    # 使用glob获取文件路径并添加到列表中
    current_month_files = glob.glob(current_month_path)
    L0.extend(current_month_files)
    
    # 将日期递增一个月
    current_month_files = glob.glob(current_month_path2)
    L1.extend(current_month_files)    
    
    start_date += timedelta(days=32)


# 对文件路径进行排序
L0.sort()

L1.sort()
# 输出文件路径列表
print(len(L0))
# 输出文件路径列表
print(len(L1))



123
123


In [26]:
#,bottom_top=3

global_regridder = None
DS=xr.concat([ext(l0,l1,lon0,lon1,lat0,lat1,bottom_top=3) for [l0,l1] in zip(L0[:],L1[:])],dim='Time')

/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070300_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070400_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070400_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070500_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070500_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070800_physical_

/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090300_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090400_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090400_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090500_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090500_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090800_physical_

In [27]:
DS.to_netcdf('/exports/d4/xuejingkai/atmos/project2024/wrf_extract/liuchao/v2/neimenggu_202207_202210.nc')

In [42]:
import pandas as pd

# 原始数据
data = {
    'Loc': ['泰安', '常州', '内蒙古棋盘井', '宿迁', '彭州', '泰安', '上海'],
    'Start_Time': ['2021.7.10', '2021.11.13', '2022.7.26', '2022.1.12', '2022.10.14', '2021.7.10', '2021.11.1'],
    'End_Time': ['2021.8.10', '2021.12.13', '2022.10.26', '2022.2.12', '2022.11.14', '2021.8.10', '2021.11.30'],
    'Lat': [36.201944, 31.811111, 39.375556, 33.964472, 31.015, 35.95, 30.785],
    'Lon': [117.086944, 119.973889, 107.012222, 118.269694, 103.945, 117.1175, 121.3225]
}

# 创建DataFrame
df = pd.DataFrame(data)

# 转换为 datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%Y.%m.%d')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%Y.%m.%d')



In [43]:
df

,Loc,Start_Time,End_Time,Lat,Lon
0,泰安,2021-07-10,2021-08-10,36.201944,117.086944
1,常州,2021-11-13,2021-12-13,31.811111,119.973889
2,内蒙古棋盘井,2022-07-26,2022-10-26,39.375556,107.012222
3,宿迁,2022-01-12,2022-02-12,33.964472,118.269694
4,彭州,2022-10-14,2022-11-14,31.015000,103.945000
5,泰安,2021-07-10,2021-08-10,35.950000,117.117500
6,上海,2021-11-01,2021-11-30,30.785000,121.322500


In [45]:
for n in range(len(df)):
    print(n)
    
    from datetime import datetime, timedelta

    # 初始日期
    start_date = df['Start_Time'][n]#datetime.strptime('2022-07-01', '%Y-%m-%d')

    # 结束日期
    end_date = df['End_Time'][n]#datetime.strptime('2022-10-31', '%Y-%m-%d')
    print(start_date,end_date)
    # 存储文件路径的列表
    L0 = []
    L1 = []
    # 循环获取每个月的文件路径
    for sdate in pd.date_range(start_date,end_date+timedelta(days=32),freq='1M'):
        current_month = sdate.strftime('%YM%m')
        current_month_path = f'/exports/d1/wrf_only_daily/fnl1.0/{current_month}/*_physical_0.nc'
        current_month_path2 = f'/exports/d1/wrf_only_daily/fnl1.0/{current_month}/*_physical_1.nc'

        # 使用glob获取文件路径并添加到列表中
        current_month_files = glob.glob(current_month_path)
        L0.extend(current_month_files)

        # 将日期递增一个月
        current_month_files = glob.glob(current_month_path2)
        L1.extend(current_month_files)    

        #start_date += timedelta(days=32)
        


    # 对文件路径进行排序
    L0.sort()

    L1.sort()
    # 输出文件路径列表
    print(len(L0))
    # 输出文件路径列表
    print(len(L1))
    print(L1[0],L1[-1])
    
    print(df['Loc'][n])

    #,bottom_top=3

    global_regridder = None
    DS=xr.concat([ext(l0,l1,lon0,lon1,lat0,lat1,bottom_top=3) for [l0,l1] in zip(L0[:],L1[:])],dim='Time')
    DS.to_netcdf(f'/exports/d4/xuejingkai/atmos/project2024/wrf_extract/liuchao/v2/{df["Loc"][n]}.nc')

0
2021-07-10 00:00:00 2021-08-10 00:00:00
62
62
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_1.nc
泰安
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070300_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070400_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070400_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070500_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070500_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070600_physical_1.nc
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_1.nc
1
2021-11-13 00:00:00 2021-12-13 00:00:00
61
61
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_1.nc
常州
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110300_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110400_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110400_physical_1.nc
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_1.nc
2
2022-07-26 00:00:00 2022-10-26 00:00:00
123
123
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022103100_physical_1.nc
内蒙古棋盘井
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M07/2022070300_physical_1.nc
/expo

/exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082800_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M08/2022082900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M08/2022083000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M08/2022083000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M08/2022083100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M08/2022083100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M09/2022090300_physical_

/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022102800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022102800_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022102900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022102900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022103000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022103000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022103100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022103100_physical_1.nc
3
2022-01-12 00:00:00 2022-02-12 00:00:00
59
59
/exports/d1/wrf_only_daily/fnl1.0/2022M01/2022010100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022800_physical_1.nc
宿迁
/exports/d1/wrf_only_daily/fnl1.0/2022M01/2022010100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M01/2022010100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M01/2022010200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M01/2022010200_physical_1.nc
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M02/2022022800_physical_1.nc
4
2022-10-14 00:00:00 2022-11-14 00:00:00
61
61
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2022M11/2022113000_physical_1.nc
彭州
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100200_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100300_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M10/2022100300_physical_1.nc
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112800_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M11/2022112900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2022M11/2022113000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2022M11/2022113000_physical_1.nc
5
2021-07-10 00:00:00 2021-08-10 00:00:00
62
62
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_1.nc
泰安
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070100_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M07/2021070200_physical_1.nc
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082800_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021082900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M08/2021083100_physical_1.nc
6
2021-11-01 00:00:00 2021-11-30 00:00:00
61
61
/exports/d1/wrf_only_daily/fnl1.0/2021M11/2021110100_physical_1.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_1.nc
上海
/exports/d1

/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122600_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122600_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122700_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122700_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122800_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122800_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122900_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021122900_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123000_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123000_physical_1.nc
/exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_0.nc /exports/d1/wrf_only_daily/fnl1.0/2021M12/2021123100_physical_1.nc
